<a href="https://colab.research.google.com/github/HanlynnKe/nlp-demo/blob/master/chinese_text_binary_classification_with_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Base code borrowed from [this Google Colab Notebook](https://colab.research.google.com/github/google-research/bert/blob/master/predicting_movie_reviews_with_bert_on_tf_hub.ipynb).

Refactored by [Hanlynn Ke](https://github.com/HanlynnKe)

Credit to [Shuyi Wang](https://www.linkedin.com/in/shuyi-wang-b3955026/)

Please refer to [this GitHub repository](https://github.com/wshuyi/demo-chinese-text-binary-classification-with-bert) for the tutorial on how to do binary classification on Chinese text data.



In [1]:
!pip install bert-tensorflow

     |████████████████████████████████| 71kB 6.2MB/s 


In [2]:
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import pickle
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

In [0]:
def pretty_print(result):
    df = pd.DataFrame([result]).T
    df.columns = ["values"]
    return df

In [0]:
def create_tokenizer_from_hub_module(bert_model_hub):
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(bert_model_hub)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

def make_features(dataset, label_list, MAX_SEQ_LENGTH, 
                  tokenizer, DATA_COLUMN, LABEL_COLUMN):
    input_example = dataset.apply(lambda x: bert.run_classifier.InputExample(guid=None,
                                                   text_a = x[DATA_COLUMN],
                                                   text_b = None,
                                                   label = x[LABEL_COLUMN]),
                                                   axis = 1)
    features = bert.run_classifier.convert_examples_to_features(input_example, label_list, MAX_SEQ_LENGTH, tokenizer)
    return features

def create_model(bert_model_hub, is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  """Creates a classification model."""

  bert_module = hub.Module(
      bert_model_hub,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)

# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(bert_model_hub, num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        bert_model_hub, is_predicting, input_ids, input_mask, 
          segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        f1_score = tf.contrib.metrics.f1_score(
            label_ids,
            predicted_labels)
        auc = tf.metrics.auc(
            label_ids,
            predicted_labels)
        recall = tf.metrics.recall(
            label_ids,
            predicted_labels)
        precision = tf.metrics.precision(
            label_ids,
            predicted_labels) 
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        return {
            "eval_accuracy": accuracy,
            "f1_score": f1_score,
            "auc": auc,
            "precision": precision,
            "recall": recall,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
        }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        bert_model_hub, is_predicting, input_ids, 
          input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn

def estimator_builder(bert_model_hub, OUTPUT_DIR, SAVE_SUMMARY_STEPS, 
                      SAVE_CHECKPOINTS_STEPS, label_list, LEARNING_RATE, 
                      num_train_steps, num_warmup_steps, BATCH_SIZE):

    # Specify outpit directory and number of checkpoint steps to save
    run_config = tf.estimator.RunConfig(
        model_dir=OUTPUT_DIR,
        save_summary_steps=SAVE_SUMMARY_STEPS,
        save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

    model_fn = model_fn_builder(
      bert_model_hub = bert_model_hub,
      num_labels=len(label_list),
      learning_rate=LEARNING_RATE,
      num_train_steps=num_train_steps,
      num_warmup_steps=num_warmup_steps)

    estimator = tf.estimator.Estimator(
      model_fn=model_fn,
      config=run_config,
      params={"batch_size": BATCH_SIZE})
    return estimator, model_fn, run_config


In [0]:
def run_on_dfs(train, test, DATA_COLUMN, LABEL_COLUMN, 
               MAX_SEQ_LENGTH = 128,
               BATCH_SIZE = 32,
               LEARNING_RATE = 2e-5,
               NUM_TRAIN_EPOCHS = 3.0,
               WARMUP_PROPORTION = 0.1,
               SAVE_SUMMARY_STEPS = 100,
               SAVE_CHECKPOINTS_STEPS = 10000,
               bert_model_hub = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"):

    label_list = train[LABEL_COLUMN].unique().tolist()
    
    tokenizer = create_tokenizer_from_hub_module(bert_model_hub)

    train_features = make_features(train, label_list, MAX_SEQ_LENGTH, tokenizer, DATA_COLUMN, LABEL_COLUMN)
    test_features = make_features(test, label_list, MAX_SEQ_LENGTH, tokenizer, DATA_COLUMN, LABEL_COLUMN)

    num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
    num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

    estimator, model_fn, run_config = estimator_builder(
                                  bert_model_hub, 
                                  OUTPUT_DIR, 
                                  SAVE_SUMMARY_STEPS, 
                                  SAVE_CHECKPOINTS_STEPS, 
                                  label_list, 
                                  LEARNING_RATE, 
                                  num_train_steps, 
                                  num_warmup_steps, 
                                  BATCH_SIZE)

    train_input_fn = bert.run_classifier.input_fn_builder(
        features=train_features,
        seq_length=MAX_SEQ_LENGTH,
        is_training=True,
        drop_remainder=False)

    estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)

    test_input_fn = run_classifier.input_fn_builder(
        features=test_features,
        seq_length=MAX_SEQ_LENGTH,
        is_training=False,
        drop_remainder=False)

    result_dict = estimator.evaluate(input_fn=test_input_fn, steps=None)
    return result_dict, estimator
    

In [0]:
import random
random.seed(10)

In [0]:
OUTPUT_DIR = 'output'

----- you just need to focus from here ------

## Get your data

In [10]:
!wget https://github.com/HanlynnKe/nlp-demo/raw/master/safety_train_valid.pickle
!wget https://github.com/HanlynnKe/nlp-demo/raw/master/test_new.csv

--2019-10-23 06:03:53--  https://github.com/HanlynnKe/nlp-demo/raw/master/safety_train_valid.pickle
Resolving github.com (github.com)... 140.82.118.3
Connecting to github.com (github.com)|140.82.118.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/HanlynnKe/nlp-demo/master/safety_train_valid.pickle [following]
--2019-10-23 06:03:53--  https://raw.githubusercontent.com/HanlynnKe/nlp-demo/master/safety_train_valid.pickle
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1181866 (1.1M) [application/octet-stream]
Saving to: ‘safety_train_valid.pickle’

safety_train_valid. 100%[===================>]   1.13M  --.-KB/s    in 0.03s   

2019-10-23 06:03:54 (33.1 MB/s) - ‘safety_train_valid.pickle’ saved [1181

In [0]:
with open("safety_train_valid.pickle", 'rb') as f:
    train, test = pickle.load(f)

In [0]:
train = train.sample(len(train))

In [13]:
train.head()

,label,comment
2550,0,蛋糕很漂亮，包装的也挺不错的，价格比较便宜
807,0,这个真的是老顾客了，分量，味道嗯不错
7448,0,不错 味道很好 和成都的火锅味道是有区别 来重庆的第一顿 炒饭超赞 吃得很满意
3877,0,菜品不错，鱼很鲜嫩，服务员态度很好，分量很足没有因为是团购就偷工减料。下次还会光顾！
8918,0,看起不错，吃起来更是太好吃了，以后可以经常来了。关键是店的装修风格我很喜欢，整体不错，喜欢！


In [0]:
myparam = {
        "DATA_COLUMN": "comment",
        "LABEL_COLUMN": "label",
        "LEARNING_RATE": 2e-5,
        "NUM_TRAIN_EPOCHS":3,
        "bert_model_hub":"https://tfhub.dev/google/bert_chinese_L-12_H-768_A-12/1"
    }

In [15]:
result, estimator = run_on_dfs(train, test, **myparam)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Writing example 0 of 8000


INFO:tensorflow:Writing example 0 of 8000


INFO:tensorflow:*** Example ***


...

INFO:tensorflow:Finished evaluation at 2019-10-23-06:26:01


INFO:tensorflow:Saving dict for global step 750: auc = 0.9429062, eval_accuracy = 0.9735, f1_score = 0.9051877, false_negatives = 28.0, false_positives = 25.0, global_step = 750, loss = 0.13678084, precision = 0.91007197, recall = 0.9003559, true_negatives = 1694.0, true_positives = 253.0


INFO:tensorflow:Saving dict for global step 750: auc = 0.9429062, eval_accuracy = 0.9735, f1_score = 0.9051877, false_negatives = 28.0, false_positives = 25.0, global_step = 750, loss = 0.13678084, precision = 0.91007197, recall = 0.9003559, true_negatives = 1694.0, true_positives = 253.0


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 750: output/model.ckpt-750


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 750: output/model.ckpt-750


In [16]:
pretty_print(result)

,values
auc,0.942906
eval_accuracy,0.973500
f1_score,0.905188
false_negatives,28.000000
false_positives,25.000000
global_step,750.000000
loss,0.136781
precision,0.910072
recall,0.900356
true_negatives,1694.000000


In [0]:
def getPrediction(in_sentences, estimator):
  labels = ["Negaitive", "Positive"]
  label_list = train['label'].unique().tolist()
  MAX_SEQ_LENGTH = 128
  bert_model_hub = "https://tfhub.dev/google/bert_chinese_L-12_H-768_A-12/1"
  
  tokenizer = create_tokenizer_from_hub_module(bert_model_hub)
  input_examples = [bert.run_classifier.InputExample(guid=None, 
                                                     text_a=x, 
                                                     text_b=None, 
                                                     label=0) 
                    for x in in_sentences]
  input_features = bert.run_classifier.convert_examples_to_features(input_examples, 
                                                                    label_list, 
                                                                    MAX_SEQ_LENGTH, 
                                                                    tokenizer)
  predict_input_fn = bert.run_classifier.input_fn_builder(
      features=input_features, 
      seq_length=MAX_SEQ_LENGTH, 
      is_training=False, 
      drop_remainder=False)

  predictions = estimator.predict(predict_input_fn)
  
  return [(sentence, prediction['probabilities'], labels[prediction['labels']]) for sentence, prediction in zip(in_sentences, predictions)]

In [36]:
target = pd.read_csv('test_new.csv')
pred_sentences = target['comment'].values.tolist()
predictions = getPrediction(pred_sentences, estimator)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Writing example 0 of 2000


INFO:tensorflow:Writing example 0 of 2000


INFO:tensorflow:*** Example ***


...

INFO:tensorflow:Restoring parameters from output/model.ckpt-750


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


In [37]:
predictions

[('糯米团是我小时候的记忆了，吃起还是好吃，只是小时候的油条没有这么硬！油茶也还好！可以试试',
  array([-3.5565728e-04, -7.9418659e+00], dtype=float32),
  'Negaitive'),
 ('满满的五星好评，口味好，服务好，特别喜欢，昨天第一次买，今天就回购了，买的刨奶，店长问我加腰果还是核桃，我说随便，他又问我喜欢吃什么，我说都喜欢，然后，帅帅的店长都给我加了，超赞',
  array([-1.7689094e-04, -8.6399336e+00], dtype=float32),
  'Negaitive'),
 ('好喝！经常会再去买来喝！就是排队的人太多了',
  array([-1.6425691e-04, -8.7138748e+00], dtype=float32),
  'Negaitive'),
 ('三个人订的四人餐，菜量大没吃完，问道不错。',
  array([-2.122892e-04, -8.457751e+00], dtype=float32),
  'Negaitive'),
 ('好的一如既往，真真爱上了自助炒饭自助八宝粥自助冰粉！！！喜欢所有菜和肉，两女一男吃两份两人餐没吃完，饮料也不错！有空就要来！',
  array([-1.4089546e-04, -8.8675079e+00], dtype=float32),
  'Negaitive'),
 ('太tmd的难吃了跟狗屎一样草',
  array([-0.00765651, -4.8760314 ], dtype=float32),
  'Negaitive'),
 ('火锅味的味道比渝李记那个火锅米线的味道好吃，更有一种老火锅的感觉，很划算，但是有韭菜叶的就好了，更喜欢吃韭菜叶的面',
  array([-1.7462636e-04, -8.6532660e+00], dtype=float32),
  'Negaitive'),
 ('今天肯定是走着来的，饿坏宝宝了',
  array([-5.5833964e-04, -7.4908547e+00], dtype=float32),
  'Negaitive'),
 ('差评。  饭是馊的！根本吃不下，再也不会点了！菜没问题，但是不可能不吃饭啊，被迫去买

In [0]:
target['label'] = None
for row in range(len(predictions)):
    if predictions[row][2] == 'Positive':
        target.loc[row, 'label'] = 1
    else:
        target.loc[row, 'label'] = 0    

In [39]:
target.head()

,id,comment,label
0,0011f384-9e54-4fb4-a272-330a6cab6804,糯米团是我小时候的记忆了，吃起还是好吃，只是小时候的油条没有这么硬！油茶也还好！可以试试,0
1,00223e4f-47e1-4fc8-9657-06444a7de9a5,满满的五星好评，口味好，服务好，特别喜欢，昨天第一次买，今天就回购了，买的刨奶，店长问我加腰...,0
2,00225350-c169-435c-84cf-970068df5b12,好喝！经常会再去买来喝！就是排队的人太多了,0
3,00a3190c-90c1-44c3-b809-7a9b1314cd27,三个人订的四人餐，菜量大没吃完，问道不错。,0
4,00b3f76e-fda3-42cd-8884-25e03a5dba64,好的一如既往，真真爱上了自助炒饭自助八宝粥自助冰粉！！！喜欢所有菜和肉，两女一男吃两份两人餐...,0


In [0]:
target_o = target[['id', 'label']]
target_o.to_csv('output_BERT.csv', index=False)